## Grouby queries from h2o-benchmarks (parquet)
## Memory Usage Dask==2022.9.1, with AMM

In [60]:
import os

import coiled
import dask.dataframe as dd
from dask.distributed import Client, performance_report
from distributed.diagnostics import MemorySampler
import pandas as pd

In [61]:
cluster = coiled.Cluster(
    name="h2o-benchmarks",
    n_workers=10,
    worker_vm_types=["t3.large"],  # 2CPU, 8GiB
    scheduler_vm_types=["t3.large"],
    package_sync=True,
)

╭───────────────────────────────────────── Package Issues ─────────────────────────────────────────╮
│                    ╷                                                           ╷                 │
│   Package          │ Issue                                                     │ Risk Level      │
│ ╶──────────────────┼───────────────────────────────────────────────────────────┼───────────────╴ │
│   libgfortran5     │ 11.3.0 has no install candidate for linux-64              │                 │
│   libgfortran      │ 5.0.0 has no install candidate for linux-64               │                 │
│   grpcio           │ 1.46.3 has no install candidate for linux-64              │                 │
│   grpc-cpp         │ 1.46.3 has no install candidate for linux-64              │                 │
│   arrow-cpp        │ 8.0.0 has no install candidate for linux-64               │                 │
│   openssl          │ Package ignored                                           │                 │
│   abseil-cpp       │ Package ignored                                           │                 │
│                    ╵                                                           ╵                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [66]:
client = Client(cluster)
client.amm.start()
client

<Client: 'tls://10.0.15.59:8786' processes=9 threads=18, memory=64.34 GiB>

In [67]:
data_size = {
    "05GB": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2_parquet/*.parquet",
    "5GB": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet/*.parquet",
    "50GB": "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2_parquet/*.parquet",
}

In [68]:
ds = "50GB" # choose "05GB" , "5GB" or "50GB"
# report_dir = "performance-reports-2"

In [69]:
ddf = dd.read_parquet(
    data_size[ds],
    engine="pyarrow",
    storage_options={"anon": True},
)
ddf

,id1,id2,id3,id4,id5,id6,v1,v2,v3
npartitions=2000,,,,,,,,,
,category[unknown],category[unknown],category[unknown],Int32,Int32,Int32,Int32,Int32,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [70]:
ms = MemorySampler()

### Q1

In [71]:
with ms.sample("q1_sept_release"):
    ddf_q1 = ddf.groupby(
        "id1", dropna=False, observed=True
    ).agg({"v1": "sum"}).compute()

In [72]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=8 threads=16, memory=57.17 GiB>

In [73]:
client.amm.start()

### Q2

In [74]:
with ms.sample("q2_sept_release"):    
    ddf_q2 = (ddf.groupby(["id1", "id2"], dropna=False, observed=True)
                .agg({"v1": "sum"})
                .compute()
             )

In [75]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=0 threads=0, memory=0 B>

In [76]:
client.amm.start()

### Q3 

Unable to run Q3 at all in June release due to high cardinality GroupBy issues.  However, in 2022.9.1 release, split_out > 1
defaults to shuffle-based GroupBy

In [77]:
ddf['id3'] = ddf['id3'].astype(str)

In [79]:
# Original query 3
with ms.sample("q3_sept_release"):
    ddf_q3 = (
        ddf.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"}, split_out=5)
        .compute()
    )

In [80]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=0 threads=0, memory=0 B>

In [81]:
client.amm.start()

### Q4

In [82]:
with ms.sample("q4_sept_release"):
    ddf_q4 = (
        ddf.groupby("id4", dropna=False, observed=True)
        .agg({"v1": "mean", "v2": "mean", "v3": "mean"}, split_out=4)
        .compute()
    )

In [83]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=6 threads=12, memory=42.88 GiB>

In [84]:
client.amm.start()

### Q5

In [85]:
with ms.sample("q5_sept_release"):
    ddf_q5 =(
        ddf.groupby("id6", dropna=False, observed=True)
        .agg({"v1": "sum", "v2": "sum", "v3": "sum"}, split_out=4)
        .compute()
    )

In [86]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=7 threads=14, memory=50.03 GiB>

In [87]:
client.amm.start()

In [88]:
### Q6 -- Unable to run in June or Sept release b/c of median not implemented

### Q7 

In [89]:
with ms.sample("q7_sept_release"):
    ddf_q7 = (
        ddf.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "max", "v2": "min"}, split_out=4)
        .assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]]
        .compute()
    )

In [101]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=9 threads=18, memory=64.34 GiB>

In [102]:
client.amm.start()

### Q8

In [103]:
with ms.sample("q8_sept_release"):    
    ddf_q8 = ddf[["id6", "v1", "v2", "v3"]]
    (
        ddf_q8[~ddf_q8["v3"].isna()][["id6", "v3"]]
        .groupby("id6", dropna=False, observed=True)
        .apply(
            lambda x: x.nlargest(2, columns="v3"),
            meta={"id6": "Int64", "v3": "float64"},
        )[["v3"]]
        .compute()
    )

In [104]:
client.restart()

<Client: 'tls://10.0.15.59:8786' processes=9 threads=18, memory=64.34 GiB>

In [105]:
client.amm.start()

### Q9

In [106]:
# try:
with ms.sample("q9_sept_release"):
    ddf_q9 = ddf[["id2", "id4", "v1", "v2"]]
    (
        ddf_q9[["id2", "id4", "v1", "v2"]]
        .groupby(["id2", "id4"], dropna=False, observed=True)
        .apply(
            lambda x: pd.Series({"r2": x.corr()["v1"]["v2"] ** 2}),
            meta={"r2": "float64"},
        )
        .compute()
    )

In [110]:
ms.to_pandas(align=True).to_csv("sept_2022.csv")

2022-09-20 11:36:54,970 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-09-20 11:36:54,974 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-09-20 11:39:07,364 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/greghayes/mambaforge/envs/coiled-runtime/lib/python3.10/site-packages/distributed/comm/tcp.py", line 491, in connect
    stream = await self.client.connect(
  File "/Users/greghayes/mambaforge/envs/coiled-runtime/lib/python3.10/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/greghayes/mambaforge/envs/coiled-runtim